### Домашка 2

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
    
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

#### Код для HH.ru

In [99]:
def parse_offer_hh(offer_text):
        
    if offer_text is None:
        return (np.nan, np.nan, np.nan)
    
    offer_text = offer_text.replace(chr(160), '')
    offer_text = offer_text.replace(chr(32), '')
    offer_text = offer_text.replace('.', '')
    
    digits = '1234567890'
    
    #парсим вида 3000-4000руб
    if offer_text[0] in digits:
        sep_pos = offer_text.find('-')
        min_offer = int(offer_text[:sep_pos])
        txt = offer_text[sep_pos+1:]
        
        for i in range(0, len(txt)):
            if txt[i] not in digits:
                sep_pos = i
                break
                
        max_offer = int(txt[:sep_pos])
        currency = txt[sep_pos:]
        
        return (min_offer, max_offer, currency)

    
    #парсим вида от3000руб
    if offer_text[:2] == 'от':
        txt = offer_text[2:]  
        
        for i in range(0, len(txt)):
            if txt[i] not in digits:
                sep_pos = i
                break
                
        min_offer = int(txt[:sep_pos])
        max_offer = np.nan
        currency = txt[sep_pos:]
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида до3000руб
    if offer_text[:2] == 'до':
        txt = offer_text[2:]  
        
        for i in range(0, len(txt)):
            if txt[i] not in digits:
                sep_pos = i
                break
                
        min_offer = np.nan
        max_offer = int(txt[:sep_pos])
        currency = txt[sep_pos:]
        
        return (min_offer, max_offer, currency)
    
    
    #если остался вариант, который мы не учли, заносим его в currency
    return (np.nan,np.nan,offer_text)


In [111]:
def parse_hh(search_request):

     
    vacancies = pd.DataFrame()
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    domain_url = 'https://novosibirsk.hh.ru'
    #local_url = '/search/vacancy?area=113&clusters=true&enable_snippets=true&text=data+science&schedule=remote'
    local_url = '/search/vacancy'
    
    params = {'area' : '113',
              'clusters' : 'true',
              'enable_snippets' : 'true',
              'text' : search_request,
              'schedule' : 'remote'
             }

    is_last_page = False
    while not is_last_page:
        
        response = requests.get(domain_url + local_url, headers=headers, params=params).text      
        soup = bs(response, 'lxml')

        vacancys_list = soup.find_all('div', {'class':'vacancy-serp-item'})

        for vacancy in vacancys_list:

            tittle_tag = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
            tittle = tittle_tag.getText().strip()  
            href = tittle_tag['href']

            company = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).getText().strip()

            offer_tag = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
            offer_str = None if offer_tag == None else offer_tag.getText()

            city = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText()

            date = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-date'}).getText()

            offer_min, offer_max, currency = parse_offer_hh(offer_str)

            temp_df = pd.DataFrame({'tittle' : [tittle],
                                    'site': ['hh.ru'],
                                    'company' : [company],
                                    'offer_min' : [offer_min],
                                    'offer_max' : [offer_max],
                                    'currency' : [currency],
                                    'href' : [href],
                                    'city' : [city],
                                    'date' : [date]
                                   })
            vacancies = vacancies.append(temp_df) 


        next_page_tag = soup.find('a', {'data-qa':'pager-next'})
        if next_page_tag == None:
            is_last_page = True
        else:
            local_url = next_page_tag['href']
            
    
    return vacancies


In [112]:
vacancies = parse_hh(search_request)

#### Код для Superjob

In [71]:
def parse_offer_superjob(offer_text):
        

    if offer_text == 'По договорённости':
        return (np.nan, np.nan, np.nan)
    
    digits = '1234567890'
    
    offer_text = offer_text.replace(chr(160), '')
    offer_text = offer_text.replace('руб./месяц', '')    

    
    #парсим вида 3000-4000
    if offer_text[0] in digits:
        sep_pos = offer_text.find('—')
        
        min_offer = int(offer_text[:sep_pos])
        max_offer = offer_text[sep_pos+1:]
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида от3000
    if offer_text[:2] == 'от':
        txt = offer_text[2:]  
                
        min_offer = int(txt)
        max_offer = np.nan
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида до3000
    if offer_text[:2] == 'до':
        txt = offer_text[2:]  
                
        min_offer = np.nan
        max_offer = int(txt)
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида 3000
    if offer_text[0] in digits:
        
        min_offer = int(offer_text)
        max_offer = int(offer_text)
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #если остался вариант, который мы не учли, заносим его в currency
    return (np.nan,np.nan,offer_text)


In [141]:
def parse_superjob(search_link):
   
    vacancies = pd.DataFrame()

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    domain_url = 'https://russia.superjob.ru/'
    

    is_last_page = False
    while not is_last_page:
        

        response = requests.get(domain_url + search_link, headers=headers).text    
        soup = bs(response, 'lxml')

        vacancys_list = soup.find_all('div', {'class':'_3zucV _1fMKr undefined _1NAsu'})

        for vacancy in vacancys_list:

            block = vacancy.find('div', {'class':'jNMYr GPKTZ _1tH7S'})
            if block == None:
                continue


            block_tittle = block.findChildren(recursive=False)[0].findChildren(recursive=False)[0]

            tittle = block_tittle.getText()
            href = block_tittle['href']

            offer_str = vacancy.find('div', {'class':'jNMYr GPKTZ _1tH7S'}).findChildren(recursive=False)[1].getText()

            block_company = vacancy.find('span', {'class':'f-test-text-vacancy-item-company-name'})
            if block_company == None:
                company = None
            else:
                company =  block_company.findChildren(recursive=False)[0].getText()

            blocks_date_city = vacancy.find('span', {'class':'f-test-text-company-item-location'})\
                                                                        .findChildren(recursive=False)

            date = blocks_date_city[0].getText()
            city = blocks_date_city[2].getText()

            offer_min, offer_max, currency = parse_offer_superjob(offer_str)


            temp_df = pd.DataFrame({'tittle' : [tittle],
                                    'site': ['superjob.ru'],
                                    'company' : [company],
                                    'offer_min' : [offer_min],
                                    'offer_max' : [offer_max],
                                    'currency' : [currency],
                                    'href' : [href],
                                    'city' : [city],
                                    'date' : [date],
                                   })
            vacancies = vacancies.append(temp_df)


        next_page_tag = soup.find('a', {'class':'f-test-button-dalshe'})
        if next_page_tag == None:
            is_last_page = True
        else:
            local_url = next_page_tag['href']

    
    return vacancies



#### Основной код

In [146]:
#поисковый запрос

search_request_hh = 'программист 1С'

# superjob  приводит запросы вида "программист 1С" и "data science" к виду "programmist-1s" и "data-scientist" 
# и вставляет их в хвост ссылки. В рамках домашки мы такое преобразование делать не будет
search_request_superjob_link = '/vakansii/programmist-1s.html?remote_work_binary=2&click_from=facet'


In [147]:
vacancies = parse_hh(search_request_hh)

vacancies = vacancies.append(parse_superjob(search_request_superjob_link))

In [148]:
vacancies.shape

(300, 9)

In [149]:
vacancies

,tittle,site,company,offer_min,offer_max,currency,href,city,date
0,Программист 1С,hh.ru,Р-Лайн (R-Line),120000.0,150000,руб,https://novosibirsk.hh.ru/vacancy/37293588?que...,"Санкт-Петербург, Лесная",25 июня
0,Программист 1С,hh.ru,ЭЛИАС,130000.0,160000,руб,https://novosibirsk.hh.ru/vacancy/37146832?que...,"Москва, ВДНХ",26 июня
0,Программист 1C (удалённая работа),hh.ru,ООО Транссертико,100000.0,NaN,руб,https://novosibirsk.hh.ru/vacancy/37675238?que...,Москва,25 июня
0,Программист 1С: Управление торговлей 11,hh.ru,ООО Рилайнс,80000.0,120000,руб,https://novosibirsk.hh.ru/vacancy/37563993?que...,Москва,27 июня
0,Программист-разработчик 1С Предприятие 8.3,hh.ru,"LR.RU, Компания",120000.0,155000,руб,https://novosibirsk.hh.ru/vacancy/37377869?que...,Москва,27 июня
...,...,...,...,...,...,...,...,...,...
0,Программист 1С,superjob.ru,Дельта Плюс,NaN,104000,руб,/vakansii/programmist-1s-33957032.html,Тула,24 июня
0,Программист 1С,superjob.ru,Дельта Плюс,NaN,104000,руб,/vakansii/programmist-1s-33957031.html,Тверь,24 июня
0,Программист 1С,superjob.ru,Дельта Плюс,NaN,104000,руб,/vakansii/programmist-1s-33957030.html,Тамбов,24 июня
0,Программист 1С,superjob.ru,Дельта Плюс,NaN,104000,руб,/vakansii/programmist-1s-33957029.html,Смоленск,24 июня


И теперь data scientist (их просто меньше удаленных)

In [152]:
#поисковый запрос

search_request_hh = 'data scientist'

search_request_superjob_link = '/vakansii/data-scientist.html?remote_work_binary=2&click_from=facet'

In [153]:
vacancies = parse_hh(search_request_hh)

vacancies = vacancies.append(parse_superjob(search_request_superjob_link))

In [154]:
vacancies.shape

(22, 9)

In [155]:
vacancies

,tittle,site,company,offer_min,offer_max,currency,href,city,date
0,Data Scientist,hh.ru,Корпорация WCH Service Bureau,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/37662353?que...,Казань,27 июня
0,Data Scientist,hh.ru,Plarium,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/36023475?que...,Краснодар,26 июня
0,Talent Acquisition Specialist (IT Recruiting),hh.ru,Inca Digital Securities,3000.0,4000.0,USD,https://novosibirsk.hh.ru/vacancy/37597197?que...,Санкт-Петербург,27 июня
0,Product Manager (ML),hh.ru,Markeaze Inc.,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/37658343?que...,Москва,27 июня
0,Virtual Assistant Analyst,hh.ru,Markeaze Inc.,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/37658350?que...,Москва,27 июня
0,Диджитал-аналитик (Digital Analyst),hh.ru,MTRENDO,75000.0,150000.0,руб,https://novosibirsk.hh.ru/vacancy/37707788?que...,"Москва, Сокол и еще 2",26 июня
0,Автор модуля NLP,hh.ru,SkillFactory,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/37705566?que...,Санкт-Петербург,26 июня
0,"Ведущий автор курса ""AI-разработчик""",hh.ru,SkillFactory,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/37705400?que...,Москва,26 июня
0,Data Scientist / Engineer (Machine Learning / ...,hh.ru,Sensoleak LTD,NaN,NaN,NaN,https://novosibirsk.hh.ru/vacancy/37662632?que...,Москва,25 июня
0,Ведущий инженер по тестированию,hh.ru,ООО UMA.Tech,NaN,250000.0,руб,https://novosibirsk.hh.ru/vacancy/36057559?que...,Москва,25 июня
